In [1]:
#https://www.tensorflow.org/text/tutorials/text_generation?hl=pt-br

import tensorflow as tf
import os
import numpy as np

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [2]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [3]:
tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))))
decode_tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))), invert=True)
tokenize_text(['a'])

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([40])>

In [4]:
def backward_text(arr):
    return tf.strings.reduce_join(decode_tokenize_text(arr), axis=-1).numpy()

In [5]:
all_tokens = tokenize_text(tf.strings.unicode_split(text, 'UTF-8'))

tensor_set = tf.data.Dataset.from_tensor_slices(all_tokens)

In [9]:
sequence_length = 100
epoch_exemple_length = len(text) // (sequence_length + 1)

# O método batch permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.

# We're gonna predict the next character using the RNN, so, to predict the word Hello, if the input is Hell, the output will be ello
dataset = tensor_set.batch(batch_size=sequence_length+1, drop_remainder=True).map(lambda sequence: [sequence[:-1], sequence[1:]])

batch_size = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
buffer_size = 10000

dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE))

vocab_size = len(tokenize_text.get_vocabulary())

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size,
    embedding_dim=256,
    rnn_units=1024)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [10]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")


checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 62ms/step - loss: 2.6691
Epoch 2/20
172/172 [==============================] - 12s 61ms/step - loss: 1.9649
Epoch 3/20
172/172 [==============================] - 12s 60ms/step - loss: 1.6867
Epoch 4/20
172/172 [==============================] - 12s 58ms/step - loss: 1.5296
Epoch 5/20
172/172 [==============================] - 12s 57ms/step - loss: 1.4349
Epoch 6/20
172/172 [==============================] - 12s 58ms/step - loss: 1.3689
Epoch 7/20
172/172 [==============================] - 12s 58ms/step - loss: 1.3171
Epoch 8/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2726
Epoch 9/20
172/172 [==============================] - 12s 60ms/step - loss: 1.2312
Epoch 10/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1913
Epoch 11/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1506
Epoch 12/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1079
E